In [10]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time

def get_date_list(start_date_str, end_date_str):
    start = datetime.strptime(start_date_str, "%Y-%m-%d")
    end = datetime.strptime(end_date_str, "%Y-%m-%d")
    dates = []
    while start <= end:
        dates.append(start.strftime("%Y%m%d"))
        start += timedelta(days=1)
    return dates

def extract_real_chart_date(soup):
    """
    벅스 차트 페이지에 실제로 표시된 날짜 추출
    <time datetime="2025.06.01 12:00"> → "20250601" 반환
    """
    time_tag = soup.select_one("fieldset.filterChart time[datetime]")
    if time_tag:
        datetime_str = time_tag["datetime"]  # 예: 2025.06.01 12:00
        chart_date = datetime.strptime(datetime_str.split()[0], "%Y.%m.%d")
        return chart_date.strftime("%Y%m%d")
    return None

def scrape_bugs_chart(date):
    url = f"https://music.bugs.co.kr/chart/track/day/total?chartdate={date}"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
     # 실제 표시된 날짜 추출
    real_chart_date = extract_real_chart_date(soup)
    if real_chart_date != date:
        print(f"요청한 날짜 {date}와 실제 차트 날짜 {real_chart_date} 불일치 → 중단")
        return None, False

    chart_table = soup.select_one("table.list.trackList.byChart tbody")
    if not chart_table:
        print(f"{date}: 차트 정보 없음")
        return [], True

    results = []
    rows = chart_table.find_all("tr")

    for row in rows:
        try:
            # 순위
            ranking = row.select_one("div.ranking > strong").text.strip()

            # 제목
            title = row.select_one("p.title").get_text(strip=True)

            # 가수
            artist = row.select_one("p.artist").get_text(strip=True)

            results.append({
                "date": date,
                "ranking": ranking,
                "title": title,
                "artist": artist
            })
        except Exception as e:
            # print(f"파싱 오류: {e}")
            continue

    print(f"{date} - {len(results)}곡 수집됨")
    return results, True

def crawl_bugs_range(start_date, end_date, delay_sec=3):
    all_results = []
    dates = get_date_list(start_date, end_date)
    for date in dates:
        print(f"{date} 크롤링 시작")
        day_result, should_continue = scrape_bugs_chart(date)
        if not should_continue:
            print("이후 날짜는 아직 업데이트되지 않음 → 중단")
            break
        all_results.extend(day_result)
        time.sleep(delay_sec)  # 서버 부하 방지

    return all_results

# 예시 실행
start_date = "2025-05-31"
end_date = "2025-06-10"

#날짜 유효성 검사
try:
    datetime.strptime(start_date, "%Y-%m-%d")
    datetime.strptime(end_date, "%Y-%m-%d")
except ValueError as e:
    print(f"날짜 형식 오류 또는 존재하지 않는 날짜입니다: {e}")
    exit(1)  # 이후 코드 실행 중단


data = crawl_bugs_range(start_date, end_date)


20250531 크롤링 시작
20250531 - 100곡 수집됨
20250601 크롤링 시작
20250601 - 100곡 수집됨
20250602 크롤링 시작
20250602 - 100곡 수집됨
20250603 크롤링 시작
요청한 날짜 20250603와 실제 차트 날짜 20250602 불일치 → 중단
이후 날짜는 아직 업데이트되지 않음 → 중단


In [11]:
from collections import defaultdict

def pretty_print_bugs_chart(data):
    # 날짜별로 정리
    chart_by_date = defaultdict(list)
    for row in data:
        chart_by_date[row["date"]].append(row)

    # 날짜 오름차순으로 출력
    for date in sorted(chart_by_date.keys()):
        print(f"\n{date} 일간 차트")
        print("-" * 50)
        # 순위순 정렬
        sorted_chart = sorted(chart_by_date[date], key=lambda x: int(x["ranking"]))
        for r in sorted_chart:
            print(f"#{r['ranking']:>3} | {r['title']} - {r['artist']}")

pretty_print_bugs_chart(data)



20250531 일간 차트
--------------------------------------------------
#  1 | THUNDER - 세븐틴(SEVENTEEN)
#  2 | Never Ending Story - 아이유(IU)
#  3 | 너에게 닿기를 - 10CM
#  4 | Drowning - WOODZ
#  5 | 네모의 꿈 - 아이유(IU)
#  6 | like JENNIE - 제니 (JENNIE)
#  7 | 그날이 오면 - 투모로우바이투게더
#  8 | HANDS UP - MEOVV (미야오)
#  9 | Whiplash - aespa
# 10 | HOT - LE SSERAFIM (르세라핌)
# 11 | Maybe Tomorrow - DAY6 (데이식스)
# 12 | 나는 반딧불 - 황가람
# 13 | REBEL HEART - IVE (아이브)
# 14 | 모르시나요(PROD.로코베리) - 조째즈
# 15 | I DO ME - KiiiKiii (키키)
# 16 | 빨간 운동화 - 아이유(IU)
# 17 | APT. - 로제(ROSÉ)로제(ROSÉ)
# 18 | Last Scene (Feat. 원슈타인) - 아이유(IU)
# 19 | 청춘만화 - 이무진
# 20 | TOO BAD (feat. Anderson .Paak) - G-DRAGON
# 21 | Pookie - FIFTY FIFTY
# 22 | HAPPY - DAY6 (데이식스)
# 23 | Die With A Smile - Lady GaGa(레이디 가가)Lady GaGa(레이디 가가)
# 24 | 한 페이지가 될 수 있게 - DAY6 (데이식스)
# 25 | toxic till the end - 로제(ROSÉ)
# 26 | 마음 따라 뛰는 건 멋지지 않아? - TWS (투어스)
# 27 | ATTITUDE - IVE (아이브)
# 28 | HOME SWEET HOME (feat. 태양, 대성) - G-DRAGON
# 29 | 오늘만 I LOVE YOU - BOYNEXTDOOR
#